In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Google Drive 연동
from google.colab import drive
drive.mount('/content/gdrive')


# quantity 열의 문자열을 숫자로 변환
def convert_quantity(quantity):
    if 'M' in quantity:
        return float(quantity.replace('M', ''))  # 백만
    elif 'B' in quantity:
        return float(quantity.replace('B', '')) * 1000  # 십억
    else:
        return float(quantity)  # 그 외의 경우는 그대로 반환


# 스케일 역변환 함수 정의
def inverse_scale(scaled_data, scaler):
    return scaler.inverse_transform(scaled_data)


# X와 y로 데이터 나누기 (시퀀스 데이터 생성)
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)


plt.rc('font', family='NanumBarunGothic')

# 데이터 불러오기
traveler_data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/bigdata/remove_city_under_5000_2.csv")
kospi_data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/bigdata/KOSPI.csv")

# 날짜 정보를 활용하여 연도와 월을 하나의 날짜 형식으로 변환
traveler_data['date'] = pd.to_datetime(traveler_data[['year', 'month']].assign(day=1))
kospi_data['date'] = pd.to_datetime(kospi_data['date'])
kospi_data['date'] = kospi_data['date'].dt.strftime('%Y-%m-%d')  # 'date' 열을 문자열로 변환

kospi_data['index'] = kospi_data['index'].str.replace(',', '').astype(float)
kospi_data['quantity'] = kospi_data['quantity'].apply(convert_quantity)

# 'date' 열을 기준으로 정렬
kospi_data = kospi_data.sort_values(by='date')

# # 도시별로 모델 학습 및 예측 수행
cities = traveler_data['city'].unique()
accuracies = []

for city in cities:
  # Kansai 지역의 여행자 데이터 선택
  city_data = traveler_data[traveler_data['city'] == city][['date', 'people']]

  print("도시 : " + city + " / 데이터 개수 : " + str(len(city_data)))

  if len(city_data) < 100 : continue

  # 'date' 열을 datetime 형식으로 변환
  city_data['date'] = pd.to_datetime(city_data['date']).dt.strftime('%Y-%m-%d')

  # 'date' 열을 기준으로 정렬
  city_data = city_data.sort_values(by='date')

  # KOSPI 데이터와 병합
  merged_data = city_data.merge(kospi_data[['date', 'index']], on='date', how='inner')

  # 데이터 스케일링 (0과 1 사이로)
  scaler = MinMaxScaler()
  scaled_data = scaler.fit_transform(merged_data[['people', 'index']])

  # 시계열 데이터를 학습 데이터와 테스트 데이터로 분할
  train_size = int(len(scaled_data) * 0.75)  # 80%를 학습 데이터로 사용
  test_size = int(len(scaled_data) * 0.9)
  train_data = scaled_data[:train_size]
  test_data = scaled_data[test_size:]

  seq_length = 12  # 시퀀스 길이 설정
  X_train, y_train = create_sequences(train_data, seq_length)
  X_test, y_test = create_sequences(test_data, seq_length)

  # LSTM 모델 정의
  model = Sequential()
  model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
  model.add(Dense(units=2))  # 출력 뉴런 수는 2로 설정
  model.compile(optimizer='adam', loss='mean_squared_error', run_eagerly=True)

  # 모델 학습
  model.fit(X_train, y_train, epochs=50, batch_size=64)

  # 테스트 데이터로 예측
  y_pred = model.predict(X_test)

  if len(y_pred) == 0 : continue

  # 테스트 데이터의 예측 결과를 스케일 역변환
  y_test_original = inverse_scale(y_test, scaler)
  y_pred_original = inverse_scale(y_pred, scaler)

  # 예측 결과를 시각화 (원래 스케일로 되돌린 값)
  plt.figure(figsize=(12, 6))
  plt.plot(merged_data['date'].iloc[test_size+seq_length:], y_test_original[:, 0], label='Actual ' + city + ' Travelers')
  plt.plot(merged_data['date'].iloc[test_size+seq_length:], y_pred_original[:, 0], label='Predicted ' + city + ' Travelers')
  plt.title(city + ' Travelers Prediction')
  plt.xlabel('Date')
  plt.ylabel('People')
  plt.legend()
  plt.show()

  # RMSE 계산
  rmse = math.sqrt(mean_squared_error(y_test_original[:, 0], y_pred_original[:, 0]))

  # MAE 계산
  mae = mean_absolute_error(y_test_original[:, 0], y_pred_original[:, 0])

  # 정확도(%) 계산 (예측 값과 실제 값의 비율로 계산)
  accuracy = 100 - (mae / np.mean(y_test_original[:, 0])) * 100
  accuracies.append(accuracy)

  print(f"RMSE: {rmse:.2f}")
  print(f"MAE: {mae:.2f}")
  print(f"정확도(%): {accuracy:.2f}%")
  print()

print(f"전체 데이터 정확도(%) :  {np.mean(accuracies)}%")